In [6]:
import pandas as pd
import ttach as tta

In [7]:
# !pip install optuna

In [8]:
import os
import shutil
import pathlib
import gc
from pathlib import Path
from PIL import Image
import pandas as pd
import numpy as np
import cv2 as cv
import random
import matplotlib.pyplot as plt
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset

import torchvision
from torchvision import datasets
import cv2
from torch.cuda import amp

import torchvision.transforms as T
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.utils import make_grid
import optuna

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp

In [9]:
def seed_everything(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [10]:
seed_everything(42)

In [11]:
# CFGS = [       
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '../../output/exp_s1_pl_02/Unet/efficientnet-b7-256/checkpoint_dice.pth',
#         'threshold': 0.12,
#         'tta': True
#     }, 

#     ### below final
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'timm-efficientnet-b8',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '../../output/exp_s1_01/Unet/timm-efficientnet-b8/checkpoint_dice.pth',
#         'threshold': 0.3,
#         'tta': True
#     }, 
    
#     {
#     'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-512/checkpoint_dice.pth',
#         'threshold': 0.1,
#         'tta': True
#     }, 
#     {
#     'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [384, 384],
#         'num_classes': 1,
#         'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-384/checkpoint_dice.pth',
#         'threshold': 0.16,
#         'tta': True
#     }, 
# #     {
# #     'model_name': 'Unet',
# #         'backbone': 'efficientnet-b7',
# #         'img_size': [256, 256],
# #         'num_classes': 1,
# #         'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-256-100-epochs/checkpoint_dice.pth',
# #         'threshold': 0.04,
# #         'tta': True
# #     },
# #     {
# #     'model_name': 'Unet',
# #         'backbone': 'efficientnet-b7',
# #         'img_size': [256, 256],
# #         'num_classes': 1,
# #         'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-256-bce-train_2/checkpoint_dice.pth',
# #         'threshold': 0.32,
# #         'tta': False
# #     },
#     {
#     'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-256-train_2/checkpoint_dice.pth',
#         'threshold': 0.32,
#         'tta': False
#     },
    
    

# ]


# CFGS = [       
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '../../output/exp_s1_pl_02/Unet/efficientnet-b7-256/checkpoint_dice.pth',
#         'threshold': 0.12,
#         'tta': False #True
#     }, 
# ]

# CFGS = [       
    
    
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail/output/exp_s1_pl_03_s2/Unet/efficientnet-b7-256/checkpoint_dice.pth',
#         'threshold': 0.24,
#         'tta': True
#     }, 
    
# #     {
# #         'model_name': 'Unet',
# #         'backbone': 'efficientnet-b7',
# #         'img_size': [256, 256],
# #         'num_classes': 1,
# #         'model_pth':  '../../output/exp_s1_pl_02/Unet/efficientnet-b7-256/checkpoint_dice.pth',
# #         'threshold': 0.12,
# #         'tta': False
# #     }, 
    
# #     {
# #         'model_name': 'Unet',
# #         'backbone': 'efficientnet-b7',
# #         'img_size': [512, 512],
# #         'num_classes': 1,
# #         'model_pth':  '/home/rohits/pv1/contrail/output/exp_s1_pl_02/Unet/efficientnet-b7/checkpoint_dice.pth',
# #         'threshold': 0.19,
# #         'tta': True
# #     }, 

    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [384, 384],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail/output/exp_s1_pl_01/Unet/efficientnet-b7-384/checkpoint_dice.pth',
#         'threshold': 0.46,
#         'tta': True
#     }, 
    
    
# #     {
# #         'model_name': 'Unet',
# #         'backbone': 'efficientnet-b7',
# #         'img_size': [256, 256],
# #         'num_classes': 1,
# #         'model_pth':  '/home/rohits/pv1/contrail/output/exp_s1_pl_01/Unet/efficientnet-b7/checkpoint_dice.pth',
# #         'threshold': 0.33,
# #         'tta': True
# #     }, 
    
    
#     ### below final
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'timm-efficientnet-b8',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '../../output/exp_s1_01/Unet/timm-efficientnet-b8/checkpoint_dice.pth',
#         'threshold': 0.3,
#         'tta': True
#     }, 
    
#     {
#     'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-512/checkpoint_dice.pth',
#         'threshold': 0.1,
#         'tta': True
#     }, 
# #     {
# #     'model_name': 'Unet',
# #         'backbone': 'efficientnet-b7',
# #         'img_size': [384, 384],
# #         'num_classes': 1,
# #         'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-384/checkpoint_dice.pth',
# #         'threshold': 0.16,
# #         'tta': True
# #     }, 
# #     {
# #     'model_name': 'Unet',
# #         'backbone': 'efficientnet-b7',
# #         'img_size': [256, 256],
# #         'num_classes': 1,
# #         'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-256-100-epochs/checkpoint_dice.pth',
# #         'threshold': 0.04,
# #         'tta': True
# #     },
# #     {
# #     'model_name': 'Unet',
# #         'backbone': 'efficientnet-b7',
# #         'img_size': [256, 256],
# #         'num_classes': 1,
# #         'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-256-bce-train_2/checkpoint_dice.pth',
# #         'threshold': 0.32,
# #         'tta': False
# #     },
#     {
#     'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '../../output/contrail_exp_b7/efficientnet-b7-256-train_2/checkpoint_dice.pth',
#         'threshold': 0.32,
#         'tta': False
#     },
    
    

# ]

In [12]:
CFGS = [           
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail/output/exp_s1_pl_03/pl_round1/efficientnet-b7-256-s2/checkpoint_dice.pth',
        'threshold': 0.62, #0.24,
        'tta': True
    },     
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail/output/exp_s1_pl_03/pl_round2/efficientnet-b7-256-s2/checkpoint_dice.pth',
        'threshold': 0.62, #0.24,
        'tta': True
    }, 
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail/output/exp_s1_pl_03/pl_round2_2/efficientnet-b7-256/checkpoint_dice.pth',
        'threshold': 0.62, #0.24,
        'tta': True
    }, 
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail/output/exp_s1_pl_03/pl_round2_2/efficientnet-b7-256-s2/checkpoint_dice.pth',
        'threshold': 0.62, #0.24,
        'tta': True
    }, 
    
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail/output/exp_s1_pl_03/pl_round2_2/efficientnet-b7-256-s2/checkpoint_dice.pth',
        'threshold': 0.62, #0.24,
        'tta': True
    },
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail/output/exp_02_pl1/Unet/efficientnet-b7-256/checkpoint_dice.pth',
#         'threshold': 0.62, #0.24,
#         'tta': True
#     },
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail/output/exp_02_pl1/Unet/efficientnet-b7-512/checkpoint_dice.pth',
        'threshold': 0.62, #0.24,
        'tta': True
    },
    
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [128, 128],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail/output/exp_fulfit_01/Unet/efficientnet-b7-128/checkpoint_dice.pth',
        'threshold': 0.62, #0.24,
        'tta': True
    },
    
    
    
    
    
    

]

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [14]:
class Dataset:
    def __init__(self, df, transform=None):   
        self.images = df['image']
        self.labels = df['label']
        self.transform =transform
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        image = np.load(self.images[idx]).astype(float)
        if  self.transform :
            data = self.transform(image=image)
            image  = data['image']
            image = np.transpose(image, (2, 0, 1))
        return torch.tensor(image)
    
    
    
class ContrailDataset:
    def __init__(self, df, transform=None):
        self.df = df  
        self.images = df['image']
        self.labels = df['label']
        self.transform =transform
        
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        image = np.load("../../input/" + self.images[idx]).astype(float)   
        label = np.load("../../input/" + self.labels[idx]).astype(float)
        
        # label_cls = 1 if label.sum() > 0 else 0
        if self.transform :
            data = self.transform(image=image, mask=label)
            image  = data['image']
            label  = data['mask']
            image = np.transpose(image, (2, 0, 1))
            label = np.transpose(label, (2, 0, 1))    
            
        return torch.tensor(image), torch.tensor(label)

In [15]:
class Net(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        self.model = smp.Unet(
            encoder_name=cfg["backbone"],      # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
            encoder_weights=None,     # use `imagenet` pre-trained weights for encoder initialization
            in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
            classes=cfg["num_classes"],        # model output channels (number of classes in your dataset)
            activation=None
        )
        
    
    def forward(self, inputs):
        mask = self.model(inputs)
        return mask

In [16]:
import torch
import torch.nn as nn


class Dice(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pred_masks: torch.Tensor, gt_masks: torch.Tensor, smooth=1e-5):
        """
        This method expect boolean pred and gt masks in the shape of (B, C, H, W).
        In this competition, we are dealing with semantic segmentation and I choose to use 
        one channel with two values: 0 for background, 1 for mask.
        """
        batch_size = pred_masks.shape[0]
        pred_masks = pred_masks.view(batch_size, -1)
        gt_masks = gt_masks.view(batch_size, -1)

        intersection = (pred_masks * gt_masks).sum(1)
        dice = (2. * intersection) / (pred_masks.sum(1) + gt_masks.sum(1) + smooth)

        dice = dice.mean()

        return dice

In [17]:
def dice_coef(y_true, y_pred, thr=0.5, epsilon=1e-6):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum()
    den = y_true.sum() + y_pred.sum()
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    
    return dice


def get_transform(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_NEAREST),
    ], p=1.0)
    return transform


In [18]:
val_df =pd.read_csv("../../input/val_df_filled.csv")
# val_df = val_df.loc[val_df['class'] == 1].reset_index(drop=True)
# val_df = val_df.loc[val_df['class'] == 0].reset_index(drop=True)


final_preds = []

for idx, cfg in enumerate(CFGS):
    print(cfg)
    val_transform = get_transform(cfg['img_size'])
    valid_dataset = ContrailDataset(val_df, transform=val_transform)  

    valid_loader = DataLoader(
        valid_dataset, 
        batch_size = 32, #32, 
        shuffle = False, 
        num_workers = 2, 
        pin_memory = True, 
        drop_last = False
    )


    model = Net(cfg)    
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    if cfg['tta']:
        model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')
    
    
    model.to(device)
    model.eval()
    preds = []
    masks_ = []
    
    for index, (images, masks) in enumerate(tqdm(valid_loader)):  
        images  = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        if cfg['img_size'][0] != 256:
            masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest') 
        masks_.append(torch.squeeze(masks, dim=1))

        with torch.inference_mode():
            pred = model(images)             
#             pred = pred.sigmoid()
        
            if cfg['img_size'][0] != 256:
                pred = torch.nn.functional.interpolate(pred, size=256, mode='nearest') 
#                 masks = torch.nn.functional.interpolate(masks, size=256, mode='bilinear') 
            preds.append(torch.squeeze(pred, dim=1))

    

    model_masks = torch.cat(masks_, dim=0)
    model_preds = torch.cat(preds, dim=0)
    
    model_masks = torch.flatten(model_masks, start_dim=0, end_dim=1)
    model_preds = torch.flatten(model_preds, start_dim=0, end_dim=1)  
    
    
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
#         if threshold > 0.55:
#             continue
        
        score = dice_coef(model_masks, model_preds.sigmoid(), thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    
        
    print(best_dice_score, best_threshold)
    final_preds.append(model_preds)
    
    del model
    torch.cuda.empty_cache()
    gc.collect()
    
    # 0.65424037

{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [256, 256], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/contrail/output/exp_s1_pl_03/pl_round1/efficientnet-b7-256-s2/checkpoint_dice.pth', 'threshold': 0.62, 'tta': True}


  0%|          | 0/58 [00:00<?, ?it/s]

0.6775598 0.16
{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [256, 256], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/contrail/output/exp_s1_pl_03/pl_round2/efficientnet-b7-256-s2/checkpoint_dice.pth', 'threshold': 0.62, 'tta': True}


  0%|          | 0/58 [00:00<?, ?it/s]

0.67928106 0.49
{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [256, 256], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/contrail/output/exp_s1_pl_03/pl_round2_2/efficientnet-b7-256/checkpoint_dice.pth', 'threshold': 0.62, 'tta': True}


...

0.6778432 0.57
{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [256, 256], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/contrail/output/exp_02_pl1/Unet/efficientnet-b7-256/checkpoint_dice.pth', 'threshold': 0.62, 'tta': True}


  0%|          | 0/58 [00:00<?, ?it/s]

0.66881436 0.27
{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [256, 256], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/contrail/output/exp_02_pl1/Unet/efficientnet-b7-512/checkpoint_dice.pth', 'threshold': 0.62, 'tta': True}


  0%|          | 0/58 [00:00<?, ?it/s]

0.518573 0.88


In [20]:
# score = dice_coef(model_masks, final_preds[0], thr=0.5).cpu().detach().numpy() # get_score(labels, preds) 
# score

In [21]:
def boost_dice(params):    
    for index, val in enumerate(params.keys()):
        if index == 0:
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    
    
#     threshold = 0.9
# #     preds1 = (nn.Sigmoid()(preds)>threshold).double()     
#     preds1 = (nn.Sigmoid()(preds)>threshold).double()            
#     score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 

    best_threshold = 0.0
    best_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
#         if threshold < 0.05 and threshold > 0.25:
#             continue
#         if threshold > 0.2:
#             continue
    
        preds1 = (nn.Sigmoid()(preds)>threshold).double()            
        score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 
        if score > best_score:
            best_score = score
            best_threshold = threshold
            
#     best_score = score
#     best_threshold = threshold
    return best_score, best_threshold

In [22]:
def objective(trial):
    params = {}    
    for i in range(len(final_preds)):
        params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1) 
        
#         params[f"w1"] = 0.5 #trial.suggest_float(f'w1', 0, 1) 
#         params[f"w2"] = 0.5 #trial.suggest_float(f'w2', 0, 1) 
#         params[f"w3"] = 0.25 # trial.suggest_float(f'w3', 0, 1) 
#         params[f"w4"] = 0.25 #trial.suggest_float(f'w4', 0, 1) 
    
        
    score, best_threshold  = boost_dice(params)
    params['threshold'] = best_threshold
    print(params)
    return score

study = optuna.create_study(direction='maximize')
# study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials=1500)

best_params = study.best_params
best_params

[I 2023-06-14 21:26:54,070] A new study created in memory with name: no-name-17ed6d22-502b-4786-ba3d-c90d8d95073b
[I 2023-06-14 21:26:55,100] Trial 0 finished with value: 0.6784290671348572 and parameters: {'w1': 0.5706063659455468, 'w2': 0.37449946739876316, 'w3': 0.4802928225523282, 'w4': 0.23580989528145513, 'w5': 0.49912656163819646, 'w6': 0.7358340649809313, 'w7': 0.45656904437576296}. Best is trial 0 with value: 0.6784290671348572.


{'w1': 0.5706063659455468, 'w2': 0.37449946739876316, 'w3': 0.4802928225523282, 'w4': 0.23580989528145513, 'w5': 0.49912656163819646, 'w6': 0.7358340649809313, 'w7': 0.45656904437576296, 'threshold': 0.36}


[I 2023-06-14 21:26:56,108] Trial 1 finished with value: 0.6787629127502441 and parameters: {'w1': 0.27429701487565494, 'w2': 0.6516481394948331, 'w3': 0.9658670674155136, 'w4': 0.7654322524824011, 'w5': 0.4870144347570854, 'w6': 0.6677452342232499, 'w7': 0.44074245046086713}. Best is trial 1 with value: 0.6787629127502441.


{'w1': 0.27429701487565494, 'w2': 0.6516481394948331, 'w3': 0.9658670674155136, 'w4': 0.7654322524824011, 'w5': 0.4870144347570854, 'w6': 0.6677452342232499, 'w7': 0.44074245046086713, 'threshold': 0.36}


[I 2023-06-14 21:26:57,117] Trial 2 finished with value: 0.6784324049949646 and parameters: {'w1': 0.0731874383797455, 'w2': 0.3645760976119339, 'w3': 0.7074812096788984, 'w4': 0.12338715821109947, 'w5': 0.37901325450763657, 'w6': 0.9522286032818058, 'w7': 0.14316065935685518}. Best is trial 1 with value: 0.6787629127502441.


...

{'w1': 0.552547798038945, 'w2': 0.7591329751754916, 'w3': 0.14354990154872338, 'w4': 0.031292229198862404, 'w5': 0.3173081243160481, 'w6': 0.018754321627066094, 'w7': 0.03624168095131215, 'threshold': 0.27}


[I 2023-06-14 21:33:31,237] Trial 380 finished with value: 0.6799482107162476 and parameters: {'w1': 0.48516106138376536, 'w2': 0.9162771971704936, 'w3': 0.048678700903860374, 'w4': 0.841279841050865, 'w5': 0.27082593965963253, 'w6': 0.4912176455803186, 'w7': 0.17978952150918845}. Best is trial 199 with value: 0.6807036399841309.


{'w1': 0.48516106138376536, 'w2': 0.9162771971704936, 'w3': 0.048678700903860374, 'w4': 0.841279841050865, 'w5': 0.27082593965963253, 'w6': 0.4912176455803186, 'w7': 0.17978952150918845, 'threshold': 0.29}


[W 2023-06-14 21:33:31,761] Trial 381 failed with parameters: {'w1': 0.9536969190594786, 'w2': 0.9433193658625045, 'w3': 0.10216559562240181, 'w4': 0.2276153754431128, 'w5': 0.20806746317174832, 'w6': 0.03445772530603394, 'w7': 0.2615863719786183} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/rohits/anaconda3/envs/contrail/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_2131012/337626026.py", line 12, in objective
    score, best_threshold  = boost_dice(params)
  File "/tmp/ipykernel_2131012/1537877821.py", line 30, in boost_dice
    score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds)
KeyboardInterrupt
[W 2023-06-14 21:33:31,762] Trial 381 failed with value None.


KeyboardInterrupt: 

In [20]:
## POS ONLY:  pl1_s2, pl2_s2, pl2_2_s1, pl2_2_s2
# 0.6885783076286316 | TH: 0.2
# {'w1': 0.6880974792134807,
#  'w2': 0.9236328771149573,
#  'w3': 0.5842894744922432,
#  'w4': 0.2397683383849518}






#th: 0.2
# {'w1': 0.9118427143017789, 'w2': 0.07722806128805348, 'w3': 0.3483576258941602}


# 0.6802493929862976
# {'w1': 0.901456237984701,
#  'w2': 0.03333500127663481,
#  'w3': 0.1012744451767772,
#  'w4': 0.3315684160625839}


# 0.6802645325660706
# {'w1': 0.9423043059931125,
#  'w2': 0.12999481950972483,
#  'w3': 0.34096541343299724}




# 0.6753096580505371
# {'w1': 0.8191544364616984,
#  'w2': 0.2172873170900641,
#  'w3': 0.9872223348463418,
#  'w4': 0.34763898831692425,
#  'w5': 0.6106201134139329}


In [ ]:
best_params

In [ ]:
# {'w1': 0.19193466652838265, 'w2': 0.0010692529855302647, 'w3': 0.19479657217496682, 'w4': 0.7062431810854415, 'threshold': 0.39}


In [ ]:
#  0.66671222448349 and parameters:
# {'w1': 0.7800660379809229, 'w2': 0.5851307264271756, 'w3': 0.5728701966485974}
# threshold: 0.32
